In [1]:
import anndata as ad
import numpy as np
import pandas as pd

# import cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

# import pearson correlation
from scipy.stats import pearsonr

In [2]:
mean_df  = pd.read_csv("/large_storage/ctc/userspace/aadduri/preprint/replogle_filtered_globalsimplesum/rpe1/eval_final.ckpt/rpe1_results.csv")
sets_df = pd.read_csv("/large_storage/ctc/userspace/aadduri/preprint/replogle_llama_21712320_filtered_copy/rpe1/eval_step=56000.ckpt/rpe1_results.csv")
sets_df['discrimination_score'].mean()

0.2917127341311134

In [31]:
adata_pred_mean = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/replogle_filtered_globalsimplesum/rpe1/eval_final.ckpt/adata_pred.h5ad')
adata_real_mean = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/replogle_filtered_globalsimplesum/rpe1/eval_final.ckpt/adata_real.h5ad')


# mean_df['discrimination_score_rev'] = compute_perturbation_ranking_score_rev(adata_pred_mean, adata_real_mean)

adata_pred = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/replogle_llama_21712320_filtered/rpe1/eval_step=56000.ckpt/adata_pred.h5ad')
adata_real = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/replogle_llama_21712320_filtered/rpe1/eval_step=56000.ckpt/adata_real.h5ad')

adata_pred_hop62 = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/tahoe_llama_58562784/holdout/eval_last.ckpt/HOP62_pred.h5ad')
adata_real_hop62 = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/tahoe_llama_58562784/holdout/eval_last.ckpt/HOP62_real.h5ad')


# sets_df['discrimination_score_rev'] = compute_perturbation_ranking_score_rev(adata_pred, adata_real)

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Assuming sets_df and mean_df are available
# You may need to adjust the effect_size_column based on your data
effect_size_column = 'de_nsig_counts_real'  # Change this to your preferred effect size column

# Combine dataframes with model labels
sets_df_labeled = sets_df.copy()
sets_df_labeled['model'] = 'sets'

mean_df_labeled = mean_df.copy()
mean_df_labeled['model'] = 'mean'

# Combine both dataframes
combined_df = pd.concat([sets_df_labeled, mean_df_labeled], ignore_index=True)

# Remove any rows with NaN values in the effect size or discrimination score columns
combined_df = combined_df.dropna(subset=[effect_size_column, 'discrimination_score'])

# Create 10 effect size bins using quantiles
combined_df['effect_size_bin'] = pd.qcut(
    combined_df[effect_size_column], 
    q=10, 
    labels=[f'Bin {i+1}' for i in range(10)],
    duplicates='drop'  # Handle cases where there are duplicate bin edges
)

# Set up the plot
fig, axes = plt.subplots(1, 10, figsize=(40, 10))
axes = axes.flatten()

# Create box plots for each bin
for i, bin_label in enumerate(combined_df['effect_size_bin'].cat.categories):
    bin_data = combined_df[combined_df['effect_size_bin'] == bin_label]
    
    # Create box plot
    sns.boxplot(
        data=bin_data, 
        x='model', 
        y='discrimination_score', 
        ax=axes[i]
    )
    
    # Customize the subplot
    axes[i].set_title(f'{bin_label}\n(n={len(bin_data)})')
    axes[i].set_xlabel('Model')
    axes[i].set_ylabel('Discrimination Score')
    axes[i].set_ylim((0, 1.0))
    
    # Add effect size range to title
    bin_min = bin_data[effect_size_column].min()
    bin_max = bin_data[effect_size_column].max()
    axes[i].set_title(f'{bin_label}\n{effect_size_column}: [{bin_min:.3f}, {bin_max:.3f}]\n(n={len(bin_data)})')

# Adjust layout
plt.tight_layout()
plt.suptitle(f'Discrimination Score Comparison by {effect_size_column} Bins', 
             fontsize=16, y=1.02)

# Show the plot
plt.show()

# Optional: Print summary statistics for each bin
print("\nSummary Statistics by Effect Size Bin:")
print("="*60)

for bin_label in combined_df['effect_size_bin'].cat.categories:
    bin_data = combined_df[combined_df['effect_size_bin'] == bin_label]
    print(f"\n{bin_label}:")
    print(f"Effect size range: [{bin_data[effect_size_column].min():.3f}, {bin_data[effect_size_column].max():.3f}]")
    
    summary = bin_data.groupby('model')['discrimination_score'].agg(['count', 'mean', 'std', 'median'])
    print(summary)

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Assuming sets_df and mean_df are available
# You may need to adjust the effect_size_column based on your data
effect_size_column = 'de_nsig_counts_real'  # Change this to your preferred effect size column

# Combine dataframes with model labels
sets_df_labeled = sets_df.copy()
sets_df_labeled['model'] = 'sets'

mean_df_labeled = mean_df.copy()
mean_df_labeled['model'] = 'mean'

# Combine both dataframes
combined_df = pd.concat([sets_df_labeled, mean_df_labeled], ignore_index=True)

# Remove any rows with NaN values in the effect size or discrimination score columns
combined_df = combined_df.dropna(subset=[effect_size_column, 'de_nsig_counts_pred'])

# Create 10 effect size bins using quantiles
combined_df['effect_size_bin'] = pd.qcut(
    combined_df[effect_size_column], 
    q=10, 
    labels=[f'Bin {i+1}' for i in range(10)],
    duplicates='drop'  # Handle cases where there are duplicate bin edges
)

# Set up the plot
fig, axes = plt.subplots(1, 10, figsize=(40, 10))
axes = axes.flatten()

# Create box plots for each bin
for i, bin_label in enumerate(combined_df['effect_size_bin'].cat.categories):
    bin_data = combined_df[combined_df['effect_size_bin'] == bin_label]
    
    # Create box plot
    sns.boxplot(
        data=bin_data, 
        x='model', 
        y='de_nsig_counts_pred', 
        ax=axes[i]
    )
    
    # Customize the subplot
    axes[i].set_title(f'{bin_label}\n(n={len(bin_data)})')
    axes[i].set_xlabel('Model')
    axes[i].set_ylabel('Discrimination Score')
    # axes[i].set_ylim((0, 1.0))
    
    # Add effect size range to title
    bin_min = bin_data[effect_size_column].min()
    bin_max = bin_data[effect_size_column].max()
    axes[i].set_title(f'{bin_label}\n{effect_size_column}: [{bin_min:.3f}, {bin_max:.3f}]\n(n={len(bin_data)})')

# Adjust layout
plt.tight_layout()
plt.suptitle(f'Discrimination Score Comparison by {effect_size_column} Bins', 
             fontsize=16, y=1.02)

# Show the plot
plt.show()

# Optional: Print summary statistics for each bin
print("\nSummary Statistics by Effect Size Bin:")
print("="*60)

for bin_label in combined_df['effect_size_bin'].cat.categories:
    bin_data = combined_df[combined_df['effect_size_bin'] == bin_label]
    print(f"\n{bin_label}:")
    print(f"Effect size range: [{bin_data[effect_size_column].min():.3f}, {bin_data[effect_size_column].max():.3f}]")
    
    summary = bin_data.groupby('model')['discrimination_score'].agg(['count', 'mean', 'std', 'median'])
    print(summary)

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

# Convert to PCA before discrim score

In [45]:
def compute_mean_perturbation_effect(adata, pert_col="gene", ctrl_pert="non-targeting"):
    adata_df = adata.to_df()
    adata_df["pert"] = adata.obs[pert_col].values
    mean_df = adata_df.groupby("pert").mean()
    mean_pert_effect = np.abs(mean_df - mean_df.loc[ctrl_pert])
    return mean_pert_effect


def compute_perturbation_id_score(adata_pred, adata_real, pert_col="gene", ctrl_pert="non-targeting"):
    ## Given a specific perturbation, identify which model-predicted
    # perturbation is most similar to it

    ## Compute true mean perturbation effect
    mean_real_effect = compute_mean_perturbation_effect(adata_real, pert_col, ctrl_pert)
    mean_pred_effect = compute_mean_perturbation_effect(adata_pred, pert_col, ctrl_pert)
    all_perts = mean_real_effect.index.values
    ## For each true perturbation effect, find the nearest neighbor predicted
    # perturbation effect

    pred_perts = []
    for pert in all_perts:
        real_effect = mean_real_effect.loc[pert].values
        pred_effects = mean_pred_effect.values
        pred_pert = all_perts[np.argmax(cosine_similarity(real_effect.reshape(1, -1), pred_effects))]
        pred_perts.append(pred_pert)

    accuracy_score = np.sum(pred_perts == all_perts) / len(all_perts)

    return accuracy_score


def compute_perturbation_ranking_score(adata_pred, adata_real, pert_col="gene", ctrl_pert="non-targeting"):
    ## Compute true mean perturbation effect
    mean_real_effect = compute_mean_perturbation_effect(adata_real, pert_col, ctrl_pert)
    mean_pred_effect = compute_mean_perturbation_effect(adata_pred, pert_col, ctrl_pert)
    all_perts = mean_real_effect.index.values

    ranks = []

    ## For each true perturbation effect, compute similarity to all predicted effects
    for pert in all_perts:
        real_effect = mean_real_effect.loc[pert].values.reshape(1, -1)
        pred_effects = mean_pred_effect.values

        # Compute cosine similarities between the real effect and all predicted effects
        similarities = cosine_similarity(real_effect, pred_effects).flatten()

        # Get the rank of the true perturbation based on similarity
        true_pert_index = np.where(all_perts == pert)[0][0]
        sorted_indices = np.argsort(similarities)[::-1]  # Sort in descending order of similarity
        rank_of_true_pert = np.where(sorted_indices == true_pert_index)[0][0]  # 1-based rank

        ranks.append(rank_of_true_pert)

    ## mean normalized rank
    mean_rank = np.mean(ranks) / len(all_perts)

    return mean_rank

def compute_perturbation_ranking_score_rev(adata_pred, adata_real, pert_col="gene", ctrl_pert="non-targeting"):
    ## Compute true mean perturbation effect
    mean_real_effect = compute_mean_perturbation_effect(adata_real, pert_col, ctrl_pert)
    mean_pred_effect = compute_mean_perturbation_effect(adata_pred, pert_col, ctrl_pert)
    all_perts = mean_pred_effect.index.values  # Changed: now iterating over predicted perturbations
    ranks = []
    ## For each predicted perturbation effect, compute similarity to all real effects
    for pert in all_perts:
        if pert == ctrl_pert:
            continue
        pred_effect = mean_pred_effect.loc[pert].values.reshape(1, -1)  # Changed: get pred effect
        real_effects = mean_real_effect.values  # Changed: compare against all real effects
        # Compute cosine similarities between the predicted effect and all real effects
        similarities = cosine_similarity(pred_effect, real_effects).flatten()
        # Get the rank of the true perturbation based on similarity
        true_pert_index = np.where(all_perts == pert)[0][0]
        sorted_indices = np.argsort(similarities)[::-1]  # Sort in descending order of similarity
        rank_of_true_pert = np.where(sorted_indices == true_pert_index)[0][0]  # 1-based rank
        ranks.append(rank_of_true_pert)
    ## mean normalized rank
    return np.array(ranks) / len(all_perts)

compute_perturbation_ranking_score(adata_pred_mean, adata_real_mean), compute_perturbation_ranking_score_rev(adata_pred_mean, adata_real_mean)

/tmp/ipykernel_3173493/2352431115.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3173493/2352431115.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3173493/2352431115.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3173493/2352431115.py:4: 

(0.24906379929476255,
 array([0.72032193, 0.83702213, 0.        , 0.01810865, 0.88732394,
        0.        , 0.00603622, 0.14486922, 0.        , 0.43863179,
        0.23742455, 0.55734406, 0.32394366, 0.25150905, 0.24547284,
        0.05030181, 0.        , 0.27162978, 0.20321932, 0.00804829,
        0.        , 0.00201207, 0.        , 0.01810865, 0.30985915,
        0.01609658, 0.2193159 , 0.61569416, 0.01810865, 0.54325956,
        0.1167002 , 0.        , 0.30382294, 0.58752515, 0.58752515,
        0.        , 0.68209256, 0.        , 0.23943662, 0.14486922,
        0.28370221, 0.19919517, 0.07042254, 0.06438632, 0.01609658,
        0.11267606, 0.10060362, 0.01609658, 0.        , 0.61971831,
        0.        , 0.31589537, 0.52917505, 0.0221328 , 0.74647887,
        0.00603622, 0.18309859, 0.02012072, 0.01408451, 0.01609658,
        0.00402414, 0.        , 0.0221328 , 0.11267606, 0.        ,
        0.16297787, 0.47887324, 0.00201207, 0.        , 0.7665996 ,
        0.0221328 , 0.0382

In [7]:
adata_pred = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/replogle_llama_21712320_filtered/rpe1/eval_step=56000.ckpt/adata_pred.h5ad')
adata_real = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/replogle_llama_21712320_filtered/rpe1/eval_step=56000.ckpt/adata_real.h5ad')

In [8]:
# create a copy but with .obsm['X_vci_1.5.2_4'] in the .X
adata_pred_emb_min = ad.AnnData(X=adata_pred.obsm['X_vci_1.5.2_4'][:, -10:], obs=adata_pred.obs)
adata_real_emb_min = ad.AnnData(X=adata_real.obsm['X_vci_1.5.2_4'][:, -10:], obs=adata_real.obs)

In [11]:
# Let's examine the basic properties of the data
print("=== DATA SHAPES ===")
print(f"adata_pred.X shape: {adata_pred.X.shape}")
print(f"adata_real.X shape: {adata_real.X.shape}")
print(f"adata_pred_emb.X shape: {adata_pred_emb.X.shape}")
print(f"adata_real_emb.X shape: {adata_real_emb.X.shape}")

print("\n=== RAW DATA STATISTICS ===")
print("PREDICTED (raw):")
print(f"  Mean: {adata_pred.X.mean():.6f}")
print(f"  Std: {adata_pred.X.std():.6f}")
print(f"  Min: {adata_pred.X.min():.6f}")
print(f"  Max: {adata_pred.X.max():.6f}")

print("REAL (raw):")
print(f"  Mean: {adata_real.X.mean():.6f}")
print(f"  Std: {adata_real.X.std():.6f}")
print(f"  Min: {adata_real.X.min():.6f}")
print(f"  Max: {adata_real.X.max():.6f}")

print("\n=== EMBEDDING STATISTICS ===")
print("PREDICTED (embeddings):")
print(f"  Mean: {adata_pred_emb.X.mean():.6f}")
print(f"  Std: {adata_pred_emb.X.std():.6f}")
print(f"  Min: {adata_pred_emb.X.min():.6f}")
print(f"  Max: {adata_pred_emb.X.max():.6f}")

print("REAL (embeddings):")
print(f"  Mean: {adata_real_emb.X.mean():.6f}")
print(f"  Std: {adata_real_emb.X.std():.6f}")
print(f"  Min: {adata_real_emb.X.min():.6f}")
print(f"  Max: {adata_real_emb.X.max():.6f}")


=== DATA SHAPES ===
adata_pred.X shape: (49389, 2000)
adata_real.X shape: (49389, 2000)
adata_pred_emb.X shape: (49389, 2058)
adata_real_emb.X shape: (49389, 2058)

=== RAW DATA STATISTICS ===
PREDICTED (raw):
  Mean: 0.459961


/home/dhruvgautam/miniconda/envs/pertsets/lib/python3.12/site-packages/numpy/core/_methods.py:152: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


  Std: inf
  Min: 0.000000
  Max: 9.234375
REAL (raw):
  Mean: 0.467529
  Std: inf
  Min: 0.000000
  Max: 7.187500

=== EMBEDDING STATISTICS ===
PREDICTED (embeddings):
  Mean: -0.000330
  Std: 0.017960
  Min: -0.935059
  Max: 0.864746
REAL (embeddings):
  Mean: -0.000328
  Std: 0.017960
  Min: -0.877930
  Max: 0.866699


In [12]:
# Let's look at some actual vectors
print("=== SAMPLE VECTORS (first 10 features) ===")
print("PREDICTED (raw) - first sample:")
print(adata_pred.X[0, :10].toarray() if hasattr(adata_pred.X, 'toarray') else adata_pred.X[0, :10])

print("\nREAL (raw) - first sample:")
print(adata_real.X[0, :10].toarray() if hasattr(adata_real.X, 'toarray') else adata_real.X[0, :10])

print("\nPREDICTED (embeddings) - first sample:")
print(adata_pred_emb.X[0, :10])

print("\nREAL (embeddings) - first sample:")
print(adata_real_emb.X[0, :10])

print("\n=== VECTOR MAGNITUDES (L2 norm) ===")
import numpy as np

# Calculate L2 norms for raw data
pred_raw_norms = np.linalg.norm(adata_pred.X.toarray() if hasattr(adata_pred.X, 'toarray') else adata_pred.X, axis=1)
real_raw_norms = np.linalg.norm(adata_real.X.toarray() if hasattr(adata_real.X, 'toarray') else adata_real.X, axis=1)

# Calculate L2 norms for embeddings
pred_emb_norms = np.linalg.norm(adata_pred_emb.X, axis=1)
real_emb_norms = np.linalg.norm(adata_real_emb.X, axis=1)

print("RAW DATA L2 norms:")
print(f"  Predicted - mean: {pred_raw_norms.mean():.6f}, std: {pred_raw_norms.std():.6f}")
print(f"  Real - mean: {real_raw_norms.mean():.6f}, std: {real_raw_norms.std():.6f}")

print("EMBEDDING L2 norms:")
print(f"  Predicted - mean: {pred_emb_norms.mean():.6f}, std: {pred_emb_norms.std():.6f}")
print(f"  Real - mean: {real_emb_norms.mean():.6f}, std: {real_emb_norms.std():.6f}")


=== SAMPLE VECTORS (first 10 features) ===
PREDICTED (raw) - first sample:
[0.4778 0.     0.     0.2445 0.     3.316  0.2078 0.5366 0.     0.    ]

REAL (raw) - first sample:
[0.    1.013 0.    0.    0.    3.092 0.    1.505 0.    0.   ]

PREDICTED (embeddings) - first sample:
[-0.002844  -0.001372   0.012566  -0.0296     0.02783   -0.03595
  0.00719    0.00575    0.0202     0.0002565]

REAL (embeddings) - first sample:
[ 0.00928  -0.003614  0.00543  -0.04504   0.01735  -0.02655   0.0176
  0.002775  0.0321    0.02696 ]

=== VECTOR MAGNITUDES (L2 norm) ===
RAW DATA L2 norms:
  Predicted - mean: 38.250000, std: inf
  Real - mean: 39.062500, std: inf
EMBEDDING L2 norms:
  Predicted - mean: 1.647461, std: inf
  Real - mean: 1.675781, std: inf


In [19]:
def compute_mean_perturbation_effect(adata, pert_col="gene", ctrl_pert="non-targeting"):
    adata_df = adata.to_df()
    adata_df["pert"] = adata.obs[pert_col].values
    mean_df = adata_df.groupby("pert").mean()
    mean_pert_effect = np.abs(mean_df - mean_df.loc[ctrl_pert])
    return mean_pert_effect

def compute_perturbation_id_score(adata_pred, adata_real, pert_col="gene", ctrl_pert="non-targeting"):
    # Compute true and predicted mean effects
    mean_real = compute_mean_perturbation_effect(adata_real, pert_col, ctrl_pert)
    mean_pred = compute_mean_perturbation_effect(adata_pred, pert_col, ctrl_pert)
    perts = mean_real.index.values

    pred_perts = []
    for pert in perts:
        real_eff = mean_real.loc[pert].values
        pred_effs = mean_pred.values
        # Euclidean distances to all predicted effects
        distances = np.linalg.norm(pred_effs - real_eff, axis=1)
        # pick the predicted perturbation with smallest distance
        pred_perts.append(perts[np.argmin(distances)])

    # accuracy
    return np.mean(np.array(pred_perts) == perts)

def compute_perturbation_ranking_score(adata_pred, adata_real, pert_col="gene", ctrl_pert="non-targeting"):
    mean_real = compute_mean_perturbation_effect(adata_real, pert_col, ctrl_pert)
    mean_pred = compute_mean_perturbation_effect(adata_pred, pert_col, ctrl_pert)
    perts = mean_real.index.values
    n = len(perts)

    ranks = []
    for i, pert in enumerate(perts):
        real_eff = mean_real.loc[pert].values
        pred_effs = mean_pred.values
        # distances
        distances = np.linalg.norm(pred_effs - real_eff, axis=1)
        # sort ascending → lowest distance = best match
        sorted_idx = np.argsort(distances)
        # find rank of the true perturbation
        rank = np.where(sorted_idx == i)[0][0]
        ranks.append(rank)

    # normalized mean rank
    return np.mean(ranks) / n

def compute_perturbation_ranking_score_rev(adata_pred, adata_real, pert_col="gene", ctrl_pert="non-targeting"):
    mean_real = compute_mean_perturbation_effect(adata_real, pert_col, ctrl_pert)
    mean_pred = compute_mean_perturbation_effect(adata_pred, pert_col, ctrl_pert)
    perts = mean_pred.index.values
    n = len(perts)

    ranks = []
    for i, pert in enumerate(perts):
        if pert == ctrl_pert:
            continue
        pred_eff = mean_pred.loc[pert].values
        real_effs = mean_real.values
        distances = np.linalg.norm(real_effs - pred_eff, axis=1)
        sorted_idx = np.argsort(distances)
        # true index in real_effs corresponds to pert’s position in mean_real.index
        true_idx = np.where(mean_real.index.values == pert)[0][0]
        rank = np.where(sorted_idx == true_idx)[0][0]
        ranks.append(rank)

    return np.array(ranks) / n

def create_cosine_similarity_matrix_of_all_reals(adata_real):
    # compute the cosine similarity between all real perturbations
    cosine_sim = np.zeros((adata_real.shape[0], adata_real.shape[0]))
    for i in range(adata_real.shape[0]):
        for j in range(adata_real.shape[0]):
            cosine_sim[i, j] = np.dot(adata_real[i], adata_real[j]) / (np.linalg.norm(adata_real[i]) * np.linalg.norm(adata_real[j]))
    return cosine_sim

In [20]:
create_cosine_similarity_matrix_of_all_reals(adata_real)

ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 32.

In [13]:
# Fix matplotlib and create distribution plots
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns

# Create subplots for distributions
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Raw data value distributions
axes[0, 0].hist(adata_pred.X.toarray().flatten() if hasattr(adata_pred.X, 'toarray') else adata_pred.X.flatten(), 
                bins=100, alpha=0.7, density=True, color='red', label='Predicted')
axes[0, 0].set_title('Raw Data Values - Predicted')
axes[0, 0].set_xlabel('Value')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_yscale('log')

axes[0, 1].hist(adata_real.X.toarray().flatten() if hasattr(adata_real.X, 'toarray') else adata_real.X.flatten(), 
                bins=100, alpha=0.7, density=True, color='blue', label='Real')
axes[0, 1].set_title('Raw Data Values - Real')
axes[0, 1].set_xlabel('Value')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_yscale('log')

# Embedding value distributions
axes[0, 2].hist(adata_pred_emb.X.flatten(), bins=100, alpha=0.7, density=True, color='red')
axes[0, 2].set_title('Embedding Values - Predicted')
axes[0, 2].set_xlabel('Value')
axes[0, 2].set_ylabel('Density')

axes[0, 3].hist(adata_real_emb.X.flatten(), bins=100, alpha=0.7, density=True, color='blue')
axes[0, 3].set_title('Embedding Values - Real')
axes[0, 3].set_xlabel('Value')
axes[0, 3].set_ylabel('Density')

# L2 norm distributions
axes[1, 0].hist(pred_raw_norms, bins=50, alpha=0.7, density=True, color='red')
axes[1, 0].set_title('L2 Norms - Raw Data Predicted')
axes[1, 0].set_xlabel('L2 Norm')
axes[1, 0].set_ylabel('Density')

axes[1, 1].hist(real_raw_norms, bins=50, alpha=0.7, density=True, color='blue')
axes[1, 1].set_title('L2 Norms - Raw Data Real')
axes[1, 1].set_xlabel('L2 Norm')
axes[1, 1].set_ylabel('Density')

axes[1, 2].hist(pred_emb_norms, bins=50, alpha=0.7, density=True, color='red')
axes[1, 2].set_title('L2 Norms - Embeddings Predicted')
axes[1, 2].set_xlabel('L2 Norm')
axes[1, 2].set_ylabel('Density')

axes[1, 3].hist(real_emb_norms, bins=50, alpha=0.7, density=True, color='blue')
axes[1, 3].set_title('L2 Norms - Embeddings Real')
axes[1, 3].set_xlabel('L2 Norm')
axes[1, 3].set_ylabel('Density')

plt.tight_layout()
plt.savefig('distribution_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved distribution plots to 'distribution_comparison.png'")


AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [15]:
# Create side-by-side comparison plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Raw data comparison
pred_raw_flat = adata_pred.X.toarray().flatten() if hasattr(adata_pred.X, 'toarray') else adata_pred.X.flatten()
real_raw_flat = adata_real.X.toarray().flatten() if hasattr(adata_real.X, 'toarray') else adata_real.X.flatten()

axes[0, 0].hist(pred_raw_flat, bins=100, alpha=0.6, density=True, color='red', label='Predicted')
axes[0, 0].hist(real_raw_flat, bins=100, alpha=0.6, density=True, color='blue', label='Real')
axes[0, 0].set_title('Raw Data Values Comparison')
axes[0, 0].set_xlabel('Value')
axes[0, 0].set_ylabel('Density')
axes[0, 0].legend()
axes[0, 0].set_yscale('log')

# Embedding comparison
axes[0, 1].hist(adata_pred_emb.X.flatten(), bins=100, alpha=0.6, density=True, color='red', label='Predicted')
axes[0, 1].hist(adata_real_emb.X.flatten(), bins=100, alpha=0.6, density=True, color='blue', label='Real')
axes[0, 1].set_title('Embedding Values Comparison')
axes[0, 1].set_xlabel('Value')
axes[0, 1].set_ylabel('Density')
axes[0, 1].legend()

# L2 norm comparison - Raw
axes[1, 0].hist(pred_raw_norms, bins=50, alpha=0.6, density=True, color='red', label='Predicted')
axes[1, 0].hist(real_raw_norms, bins=50, alpha=0.6, density=True, color='blue', label='Real')
axes[1, 0].set_title('L2 Norms - Raw Data Comparison')
axes[1, 0].set_xlabel('L2 Norm')
axes[1, 0].set_ylabel('Density')
axes[1, 0].legend()

# L2 norm comparison - Embeddings
axes[1, 1].hist(pred_emb_norms, bins=50, alpha=0.6, density=True, color='red', label='Predicted')
axes[1, 1].hist(real_emb_norms, bins=50, alpha=0.6, density=True, color='blue', label='Real')
axes[1, 1].set_title('L2 Norms - Embeddings Comparison')
axes[1, 1].set_xlabel('L2 Norm')
axes[1, 1].set_ylabel('Density')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('side_by_side_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved side-by-side comparison plots to 'side_by_side_comparison.png'")


NameError: name 'plt' is not defined

In [16]:
# Detailed percentile analysis
print("=== DETAILED PERCENTILE ANALYSIS ===")

percentiles = [0.1, 1, 5, 10, 25, 50, 75, 90, 95, 99, 99.9]

print("\nRAW DATA PERCENTILES:")
print("Predicted:")
for p in percentiles:
    val = np.percentile(pred_raw_flat, p)
    print(f"  {p:5.1f}%: {val:12.6f}")

print("Real:")
for p in percentiles:
    val = np.percentile(real_raw_flat, p)
    print(f"  {p:5.1f}%: {val:12.6f}")

print("\nEMBEDDING PERCENTILES:")
print("Predicted:")
for p in percentiles:
    val = np.percentile(adata_pred_emb.X.flatten(), p)
    print(f"  {p:5.1f}%: {val:12.6f}")

print("Real:")
for p in percentiles:
    val = np.percentile(adata_real_emb.X.flatten(), p)
    print(f"  {p:5.1f}%: {val:12.6f}")

print("\nL2 NORM PERCENTILES:")
print("Raw Data - Predicted:")
for p in percentiles:
    val = np.percentile(pred_raw_norms, p)
    print(f"  {p:5.1f}%: {val:12.6f}")

print("Raw Data - Real:")
for p in percentiles:
    val = np.percentile(real_raw_norms, p)
    print(f"  {p:5.1f}%: {val:12.6f}")

print("Embeddings - Predicted:")
for p in percentiles:
    val = np.percentile(pred_emb_norms, p)
    print(f"  {p:5.1f}%: {val:12.6f}")

print("Embeddings - Real:")
for p in percentiles:
    val = np.percentile(real_emb_norms, p)
    print(f"  {p:5.1f}%: {val:12.6f}")


=== DETAILED PERCENTILE ANALYSIS ===

RAW DATA PERCENTILES:
Predicted:


NameError: name 'pred_raw_flat' is not defined

In [17]:
# Create box plots for better visualization of distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sample random subset for box plots (to avoid memory issues)
n_sample = min(10000, len(pred_raw_flat))
idx_sample = np.random.choice(len(pred_raw_flat), n_sample, replace=False)

# Raw data box plot
raw_data_for_box = pd.DataFrame({
    'Value': np.concatenate([pred_raw_flat[idx_sample], real_raw_flat[idx_sample]]),
    'Type': ['Predicted']*n_sample + ['Real']*n_sample
})

sns.boxplot(data=raw_data_for_box, x='Type', y='Value', ax=axes[0, 0])
axes[0, 0].set_title('Raw Data Values Distribution')
axes[0, 0].set_yscale('log')

# Embedding box plot
n_sample_emb = min(10000, adata_pred_emb.X.shape[0] * adata_pred_emb.X.shape[1])
pred_emb_flat = adata_pred_emb.X.flatten()
real_emb_flat = adata_real_emb.X.flatten()
idx_sample_emb = np.random.choice(len(pred_emb_flat), n_sample_emb, replace=False)

emb_data_for_box = pd.DataFrame({
    'Value': np.concatenate([pred_emb_flat[idx_sample_emb], real_emb_flat[idx_sample_emb]]),
    'Type': ['Predicted']*n_sample_emb + ['Real']*n_sample_emb
})

sns.boxplot(data=emb_data_for_box, x='Type', y='Value', ax=axes[0, 1])
axes[0, 1].set_title('Embedding Values Distribution')

# L2 norm box plots
norm_data_raw = pd.DataFrame({
    'L2_Norm': np.concatenate([pred_raw_norms, real_raw_norms]),
    'Type': ['Predicted']*len(pred_raw_norms) + ['Real']*len(real_raw_norms)
})

sns.boxplot(data=norm_data_raw, x='Type', y='L2_Norm', ax=axes[1, 0])
axes[1, 0].set_title('L2 Norms - Raw Data')

norm_data_emb = pd.DataFrame({
    'L2_Norm': np.concatenate([pred_emb_norms, real_emb_norms]),
    'Type': ['Predicted']*len(pred_emb_norms) + ['Real']*len(real_emb_norms)
})

sns.boxplot(data=norm_data_emb, x='Type', y='L2_Norm', ax=axes[1, 1])
axes[1, 1].set_title('L2 Norms - Embeddings')

plt.tight_layout()
plt.savefig('box_plot_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved box plot comparisons to 'box_plot_comparison.png'")


NameError: name 'plt' is not defined

In [9]:
# create a copy but with .obsm['X_vci_1.5.2_4'] in the .X
adata_pred_emb = ad.AnnData(X=adata_pred.obsm['X_vci_1.5.2_4'], obs=adata_pred.obs)
adata_real_emb = ad.AnnData(X=adata_real.obsm['X_vci_1.5.2_4'], obs=adata_real.obs)

In [10]:
len(compute_perturbation_ranking_score_rev(adata_pred, adata_real, pert_col="gene", ctrl_pert="non-targeting"))

NameError: name 'compute_perturbation_ranking_score_rev' is not defined

In [32]:
# Fix the cosine similarity matrix function
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def create_cosine_similarity_matrix_efficient(adata, sample_size=None):
    """
    Create cosine similarity matrix for the data
    
    Parameters:
    -----------
    adata : AnnData
        The data to compute similarity for
    sample_size : int, optional
        If provided, randomly sample this many vectors to make computation feasible
    
    Returns:
    --------
    cosine_sim : np.ndarray
        Cosine similarity matrix
    indices : np.ndarray
        Indices of sampled vectors (if sampling was used)
    """
    # Get the data matrix
    X = adata.X.toarray() if hasattr(adata.X, 'toarray') else adata.X
    
    # Sample if requested
    if sample_size is not None and sample_size < X.shape[0]:
        print(f"Sampling {sample_size} vectors from {X.shape[0]} total vectors")
        indices = np.random.choice(X.shape[0], sample_size, replace=False)
        X_sample = X[indices]
        print(f"Sample shape: {X_sample.shape}")
    else:
        X_sample = X
        indices = np.arange(X.shape[0])
        print(f"Using all {X.shape[0]} vectors")
    
    # Compute cosine similarity matrix
    print("Computing cosine similarity matrix...")
    cosine_sim = cosine_similarity(X_sample)
    print(f"Similarity matrix shape: {cosine_sim.shape}")
    
    return cosine_sim, indices

# Let's start with a smaller sample to test
print("=== COSINE SIMILARITY ANALYSIS ===")
print(f"Real data shape: {adata_real.X.shape}")
print(f"Real embedding shape: {adata_real_emb.X.shape}")

# Start with a sample of 1000 vectors for raw data
cosine_sim_real_sample, indices_real = create_cosine_similarity_matrix_efficient(adata_real_hop62, sample_size=1000)
cosine_sim_pred_sample, indices_pred = create_cosine_similarity_matrix_efficient(adata_pred_hop62, sample_size=1000)


=== COSINE SIMILARITY ANALYSIS ===
Real data shape: (49389, 2000)
Real embedding shape: (49389, 2058)
Sampling 1000 vectors from 2123429 total vectors
Sample shape: (1000, 2000)
Computing cosine similarity matrix...
Similarity matrix shape: (1000, 1000)
Sampling 1000 vectors from 2123429 total vectors
Sample shape: (1000, 2000)
Computing cosine similarity matrix...
Similarity matrix shape: (1000, 1000)


In [33]:
# Analyze the cosine similarity matrix
print("\n=== COSINE SIMILARITY MATRIX STATISTICS ===")
print(f"Matrix shape: {cosine_sim_pred_sample.shape}")
print(f"Mean similarity: {cosine_sim_pred_sample.mean():.6f}")
print(f"Std similarity: {cosine_sim_pred_sample.std():.6f}")
print(f"Min similarity: {cosine_sim_pred_sample.min():.6f}")
print(f"Max similarity: {cosine_sim_pred_sample.max():.6f}")

# Diagonal should be 1.0 (self-similarity)
diagonal_values = np.diag(cosine_sim_pred_sample)
print(f"Diagonal values (should be ~1.0): mean={diagonal_values.mean():.6f}, std={diagonal_values.std():.6f}")

# Off-diagonal similarities (excluding self-similarity)
mask = ~np.eye(cosine_sim_pred_sample.shape[0], dtype=bool)
off_diagonal = cosine_sim_pred_sample[mask]
print(f"Off-diagonal similarities: mean={off_diagonal.mean():.6f}, std={off_diagonal.std():.6f}")

# Look at distribution of similarities
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
print(f"Off-diagonal similarity percentiles:")
for p in percentiles:
    val = np.percentile(off_diagonal, p)
    print(f"  {p:2d}%: {val:.6f}")

# Find most and least similar pairs
flat_indices = np.unravel_index(np.argsort(cosine_sim_pred_sample, axis=None), cosine_sim_pred_sample.shape)
# Exclude diagonal elements for finding extremes
off_diag_flat = cosine_sim_pred_sample[mask]
sorted_off_diag_idx = np.argsort(off_diag_flat)

print(f"\nMost similar pair (excluding self): {off_diag_flat[sorted_off_diag_idx[-1]]:.6f}")
print(f"Least similar pair: {off_diag_flat[sorted_off_diag_idx[0]]:.6f}")



=== COSINE SIMILARITY MATRIX STATISTICS ===
Matrix shape: (1000, 1000)
Mean similarity: 0.551723
Std similarity: 0.162005
Min similarity: 0.000000
Max similarity: 1.000000
Diagonal values (should be ~1.0): mean=1.000000, std=0.000000
Off-diagonal similarities: mean=0.551274, std=0.161464
Off-diagonal similarity percentiles:
   1%: 0.116880
   5%: 0.245423
  10%: 0.337756
  25%: 0.456606
  50%: 0.565094
  75%: 0.662877
  90%: 0.748588
  95%: 0.794463
  99%: 0.868696

Most similar pair (excluding self): 0.991899
Least similar pair: 0.000000


In [34]:
# Analyze the cosine similarity matrix
print("\n=== COSINE SIMILARITY MATRIX STATISTICS ===")
print(f"Matrix shape: {cosine_sim_real_sample.shape}")
print(f"Mean similarity: {cosine_sim_real_sample.mean():.6f}")
print(f"Std similarity: {cosine_sim_real_sample.std():.6f}")
print(f"Min similarity: {cosine_sim_real_sample.min():.6f}")
print(f"Max similarity: {cosine_sim_real_sample.max():.6f}")

# Diagonal should be 1.0 (self-similarity)
diagonal_values = np.diag(cosine_sim_real_sample)
print(f"Diagonal values (should be ~1.0): mean={diagonal_values.mean():.6f}, std={diagonal_values.std():.6f}")

# Off-diagonal similarities (excluding self-similarity)
mask = ~np.eye(cosine_sim_real_sample.shape[0], dtype=bool)
off_diagonal = cosine_sim_real_sample[mask]
print(f"Off-diagonal similarities: mean={off_diagonal.mean():.6f}, std={off_diagonal.std():.6f}")

# Look at distribution of similarities
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
print(f"Off-diagonal similarity percentiles:")
for p in percentiles:
    val = np.percentile(off_diagonal, p)
    print(f"  {p:2d}%: {val:.6f}")

# Find most and least similar pairs
flat_indices = np.unravel_index(np.argsort(cosine_sim_real_sample, axis=None), cosine_sim_real_sample.shape)
# Exclude diagonal elements for finding extremes
off_diag_flat = cosine_sim_real_sample[mask]
sorted_off_diag_idx = np.argsort(off_diag_flat)

print(f"\nMost similar pair (excluding self): {off_diag_flat[sorted_off_diag_idx[-1]]:.6f}")
print(f"Least similar pair: {off_diag_flat[sorted_off_diag_idx[0]]:.6f}")



=== COSINE SIMILARITY MATRIX STATISTICS ===
Matrix shape: (1000, 1000)
Mean similarity: 0.383357
Std similarity: 0.115561
Min similarity: 0.000000
Max similarity: 1.000000
Diagonal values (should be ~1.0): mean=1.000000, std=0.000000
Off-diagonal similarities: mean=0.382739, std=0.113959
Off-diagonal similarity percentiles:
   1%: 0.092539
   5%: 0.173939
  10%: 0.229450
  25%: 0.311508
  50%: 0.390172
  75%: 0.463218
  90%: 0.523439
  95%: 0.556829
  99%: 0.615974

Most similar pair (excluding self): 0.762782
Least similar pair: 0.000000


In [23]:
# Now let's do the same for embeddings (which should be faster due to lower dimensionality)
print("\n=== COSINE SIMILARITY FOR EMBEDDINGS ===")
cosine_sim_emb_sample, indices_emb = create_cosine_similarity_matrix_efficient(adata_real_emb, sample_size=2000)

print("\n=== EMBEDDING COSINE SIMILARITY STATISTICS ===")
print(f"Matrix shape: {cosine_sim_emb_sample.shape}")
print(f"Mean similarity: {cosine_sim_emb_sample.mean():.6f}")
print(f"Std similarity: {cosine_sim_emb_sample.std():.6f}")
print(f"Min similarity: {cosine_sim_emb_sample.min():.6f}")
print(f"Max similarity: {cosine_sim_emb_sample.max():.6f}")

# Off-diagonal similarities for embeddings
mask_emb = ~np.eye(cosine_sim_emb_sample.shape[0], dtype=bool)
off_diagonal_emb = cosine_sim_emb_sample[mask_emb]
print(f"Off-diagonal similarities: mean={off_diagonal_emb.mean():.6f}, std={off_diagonal_emb.std():.6f}")

print(f"Embedding off-diagonal similarity percentiles:")
for p in percentiles:
    val = np.percentile(off_diagonal_emb, p)
    print(f"  {p:2d}%: {val:.6f}")

off_diag_emb_flat = cosine_sim_emb_sample[mask_emb]
sorted_off_diag_emb_idx = np.argsort(off_diag_emb_flat)

print(f"\nMost similar embedding pair (excluding self): {off_diag_emb_flat[sorted_off_diag_emb_idx[-1]]:.6f}")
print(f"Least similar embedding pair: {off_diag_emb_flat[sorted_off_diag_emb_idx[0]]:.6f}")



=== COSINE SIMILARITY FOR EMBEDDINGS ===
Sampling 2000 vectors from 49389 total vectors
Sample shape: (2000, 2058)
Computing cosine similarity matrix...
Similarity matrix shape: (2000, 2000)

=== EMBEDDING COSINE SIMILARITY STATISTICS ===
Matrix shape: (2000, 2000)
Mean similarity: 0.891477
Std similarity: 0.034753
Min similarity: 0.630303
Max similarity: 1.000000
Off-diagonal similarities: mean=0.891423, std=0.034677
Embedding off-diagonal similarity percentiles:
   1%: 0.764490
   5%: 0.823949
  10%: 0.849600
  25%: 0.877906
  50%: 0.898541
  75%: 0.914051
  90%: 0.925813
  95%: 0.931985
  99%: 0.942137

Most similar embedding pair (excluding self): 0.970238
Least similar embedding pair: 0.630303


In [39]:
# Create Euclidean distance matrix function
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances

def create_euclidean_distance_matrix_efficient(adata, sample_size=None):
    """
    Create Euclidean distance matrix for the data
    
    Parameters:
    -----------
    adata : AnnData
        The data to compute distances for
    sample_size : int, optional
        If provided, randomly sample this many vectors to make computation feasible
    
    Returns:
    --------
    euclidean_dist : np.ndarray
        Euclidean distance matrix
    indices : np.ndarray
        Indices of sampled vectors (if sampling was used)
    """
    # Get the data matrix
    X = adata.X.toarray() if hasattr(adata.X, 'toarray') else adata.X
    
    # Sample if requested
    if sample_size is not None and sample_size < X.shape[0]:
        print(f"Sampling {sample_size} vectors from {X.shape[0]} total vectors")
        indices = np.random.choice(X.shape[0], sample_size, replace=False)
        X_sample = X[indices]
        print(f"Sample shape: {X_sample.shape}")
    else:
        X_sample = X
        indices = np.arange(X.shape[0])
        print(f"Using all {X.shape[0]} vectors")
    
    # Compute Euclidean distance matrix
    print("Computing Euclidean distance matrix...")
    euclidean_dist = manhattan_distances(X_sample)
    print(f"Distance matrix shape: {euclidean_dist.shape}")
    
    return euclidean_dist, indices

# Compute Euclidean distance matrices for the same samples
print("=== EUCLIDEAN DISTANCE ANALYSIS ===")

# Use same random seed for fair comparison
np.random.seed(42)
euclidean_dist_real_sample, _ = create_euclidean_distance_matrix_efficient(adata_real, sample_size=1000)


=== EUCLIDEAN DISTANCE ANALYSIS ===
Sampling 1000 vectors from 49389 total vectors
Sample shape: (1000, 2000)
Computing Euclidean distance matrix...
Distance matrix shape: (1000, 1000)


In [40]:
# Analyze the Euclidean distance matrix
print("\n=== EUCLIDEAN DISTANCE MATRIX STATISTICS ===")
print(f"Matrix shape: {euclidean_dist_real_sample.shape}")
print(f"Mean distance: {euclidean_dist_real_sample.mean():.6f}")
print(f"Std distance: {euclidean_dist_real_sample.std():.6f}")
print(f"Min distance: {euclidean_dist_real_sample.min():.6f}")
print(f"Max distance: {euclidean_dist_real_sample.max():.6f}")

# Diagonal should be 0.0 (self-distance)
diagonal_values_dist = np.diag(euclidean_dist_real_sample)
print(f"Diagonal values (should be ~0.0): mean={diagonal_values_dist.mean():.6f}, std={diagonal_values_dist.std():.6f}")

# Off-diagonal distances (excluding self-distance)
mask_dist = ~np.eye(euclidean_dist_real_sample.shape[0], dtype=bool)
off_diagonal_dist = euclidean_dist_real_sample[mask_dist]
print(f"Off-diagonal distances: mean={off_diagonal_dist.mean():.6f}, std={off_diagonal_dist.std():.6f}")

# Look at distribution of distances
print(f"Off-diagonal distance percentiles:")
for p in percentiles:
    val = np.percentile(off_diagonal_dist, p)
    print(f"  {p:2d}%: {val:.6f}")

# Find most and least distant pairs
sorted_off_diag_dist_idx = np.argsort(off_diagonal_dist)

print(f"\nMost distant pair: {off_diagonal_dist[sorted_off_diag_dist_idx[-1]]:.6f}")
print(f"Least distant pair (excluding self): {off_diagonal_dist[sorted_off_diag_dist_idx[0]]:.6f}")

# Also compute for embeddings
np.random.seed(42)
euclidean_dist_emb_sample, _ = create_euclidean_distance_matrix_efficient(adata_real_emb, sample_size=2000)

print("\n=== EMBEDDING EUCLIDEAN DISTANCE STATISTICS ===")
mask_emb_dist = ~np.eye(euclidean_dist_emb_sample.shape[0], dtype=bool)
off_diagonal_emb_dist = euclidean_dist_emb_sample[mask_emb_dist]
print(f"Off-diagonal distances: mean={off_diagonal_emb_dist.mean():.6f}, std={off_diagonal_emb_dist.std():.6f}")

print(f"Embedding off-diagonal distance percentiles:")
for p in percentiles:
    val = np.percentile(off_diagonal_emb_dist, p)
    print(f"  {p:2d}%: {val:.6f}")

sorted_off_diag_emb_dist_idx = np.argsort(off_diagonal_emb_dist)
print(f"\nMost distant embedding pair: {off_diagonal_emb_dist[sorted_off_diag_emb_dist_idx[-1]]:.6f}")
print(f"Least distant embedding pair (excluding self): {off_diagonal_emb_dist[sorted_off_diag_emb_dist_idx[0]]:.6f}")



=== EUCLIDEAN DISTANCE MATRIX STATISTICS ===
Matrix shape: (1000, 1000)
Mean distance: 783.013035
Std distance: 81.163833
Min distance: 0.000000
Max distance: 1202.497559
Diagonal values (should be ~0.0): mean=0.000000, std=0.000000
Off-diagonal distances: mean=783.796831, std=77.329325
Off-diagonal distance percentiles:
   1%: 608.730352
   5%: 665.031250
  10%: 692.349829
  25%: 734.325439
  50%: 779.271118
  75%: 828.544922
  90%: 880.565527
  95%: 917.891187
  99%: 1000.234805

Most distant pair: 1202.497559
Least distant pair (excluding self): 418.563965
Sampling 2000 vectors from 49389 total vectors
Sample shape: (2000, 2058)
Computing Euclidean distance matrix...
Distance matrix shape: (2000, 2000)

=== EMBEDDING EUCLIDEAN DISTANCE STATISTICS ===
Off-diagonal distances: mean=27.331081, std=3.417234
Embedding off-diagonal distance percentiles:
   1%: 20.819709
   5%: 22.538638
  10%: 23.455931
  25%: 25.007704
  50%: 26.882766
  75%: 29.149899
  90%: 31.904585
  95%: 33.818535
 

In [13]:
compute_perturbation_ranking_score(adata_pred_emb, adata_real_emb, pert_col="gene", ctrl_pert="non-targeting")

/tmp/ipykernel_3168865/238431976.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3168865/238431976.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()


0.29083150816367015

In [ ]:
# Complete the Euclidean distance function and compute for embeddings
def create_euclidean_distance_matrix_efficient(adata, sample_size=None):
    """
    Create Euclidean distance matrix for the data
    
    Parameters:
    -----------
    adata : AnnData
        The data to compute distances for
    sample_size : int, optional
        If provided, randomly sample this many vectors to make computation feasible
    
    Returns:
    --------
    euclidean_dist : np.ndarray
        Euclidean distance matrix
    indices : np.ndarray
        Indices of sampled vectors (if sampling was used)
    """
    # Get the data matrix
    X = adata.X.toarray() if hasattr(adata.X, 'toarray') else adata.X
    
    # Sample if requested
    if sample_size is not None and sample_size < X.shape[0]:
        print(f"Sampling {sample_size} vectors from {X.shape[0]} total vectors")
        indices = np.random.choice(X.shape[0], sample_size, replace=False)
        X_sample = X[indices]
        print(f"Sample shape: {X_sample.shape}")
    else:
        X_sample = X
        indices = np.arange(X.shape[0])
        print(f"Using all {X.shape[0]} vectors")
    
    # Compute Euclidean distance matrix
    print("Computing Euclidean distance matrix...")
    euclidean_dist = euclidean_distances(X_sample)
    print(f"Distance matrix shape: {euclidean_dist.shape}")
    
    return euclidean_dist, indices

# Compute Euclidean distances for embeddings
print("=== EUCLIDEAN DISTANCE FOR EMBEDDINGS ===")
euclidean_dist_emb_sample, indices_emb_dist = create_euclidean_distance_matrix_efficient(adata_real_emb, sample_size=2000)


In [ ]:
# Analyze the Euclidean distance matrix for embeddings
print("\n=== EUCLIDEAN DISTANCE MATRIX STATISTICS (EMBEDDINGS) ===")
print(f"Matrix shape: {euclidean_dist_emb_sample.shape}")
print(f"Mean distance: {euclidean_dist_emb_sample.mean():.6f}")
print(f"Std distance: {euclidean_dist_emb_sample.std():.6f}")
print(f"Min distance: {euclidean_dist_emb_sample.min():.6f}")
print(f"Max distance: {euclidean_dist_emb_sample.max():.6f}")

# Diagonal should be 0.0 (self-distance)
diagonal_values_dist_emb = np.diag(euclidean_dist_emb_sample)
print(f"Diagonal values (should be ~0.0): mean={diagonal_values_dist_emb.mean():.6f}, std={diagonal_values_dist_emb.std():.6f}")

# Off-diagonal distances (excluding self-distance)
mask_dist_emb = ~np.eye(euclidean_dist_emb_sample.shape[0], dtype=bool)
off_diagonal_dist_emb = euclidean_dist_emb_sample[mask_dist_emb]
print(f"Off-diagonal distances: mean={off_diagonal_dist_emb.mean():.6f}, std={off_diagonal_dist_emb.std():.6f}")

# Look at distribution of distances
print(f"Embedding off-diagonal distance percentiles:")
for p in percentiles:
    val = np.percentile(off_diagonal_dist_emb, p)
    print(f"  {p:2d}%: {val:.6f}")

# Find closest and farthest pairs
sorted_off_diag_dist_emb_idx = np.argsort(off_diagonal_dist_emb)

print(f"\nClosest embedding pair (excluding self): {off_diagonal_dist_emb[sorted_off_diag_dist_emb_idx[0]]:.6f}")
print(f"Farthest embedding pair: {off_diagonal_dist_emb[sorted_off_diag_dist_emb_idx[-1]]:.6f}")


In [ ]:
# Compare Cosine Similarity vs Euclidean Distance for embeddings
print("\n=== COSINE SIMILARITY vs EUCLIDEAN DISTANCE COMPARISON (EMBEDDINGS) ===")

# Calculate correlation between cosine similarity and Euclidean distance
# Note: We need to use the same sample indices for fair comparison
cosine_sim_values = cosine_sim_emb_sample[mask_emb]
euclidean_dist_values = euclidean_dist_emb_sample[mask_dist_emb]

correlation = np.corrcoef(cosine_sim_values, euclidean_dist_values)[0, 1]
print(f"Correlation between cosine similarity and Euclidean distance: {correlation:.6f}")

# Convert cosine similarity to cosine distance for better comparison
cosine_distance_values = 1 - cosine_sim_values
correlation_cosine_dist = np.corrcoef(cosine_distance_values, euclidean_dist_values)[0, 1]
print(f"Correlation between cosine distance and Euclidean distance: {correlation_cosine_dist:.6f}")

print(f"\nSummary comparison:")
print(f"Cosine similarity: mean={cosine_sim_values.mean():.6f}, std={cosine_sim_values.std():.6f}")
print(f"Cosine distance:   mean={cosine_distance_values.mean():.6f}, std={cosine_distance_values.std():.6f}")
print(f"Euclidean distance: mean={euclidean_dist_values.mean():.6f}, std={euclidean_dist_values.std():.6f}")

# Normalize Euclidean distances for comparison
euclidean_normalized = (euclidean_dist_values - euclidean_dist_values.min()) / (euclidean_dist_values.max() - euclidean_dist_values.min())
print(f"Euclidean (normalized): mean={euclidean_normalized.mean():.6f}, std={euclidean_normalized.std():.6f}")

correlation_normalized = np.corrcoef(cosine_distance_values, euclidean_normalized)[0, 1]
print(f"Correlation between cosine distance and normalized Euclidean distance: {correlation_normalized:.6f}")


In [ ]:
# Create visualization comparing cosine similarity and Euclidean distance
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Heatmap of Euclidean distance matrix (embeddings)
im1 = axes[0, 0].imshow(euclidean_dist_emb_sample, cmap='viridis', aspect='auto')
axes[0, 0].set_title('Euclidean Distance Matrix - Embeddings')
axes[0, 0].set_xlabel('Sample Index')
axes[0, 0].set_ylabel('Sample Index')
plt.colorbar(im1, ax=axes[0, 0])

# Heatmap of cosine similarity matrix (embeddings) for comparison
im2 = axes[0, 1].imshow(cosine_sim_emb_sample, cmap='viridis', aspect='auto')
axes[0, 1].set_title('Cosine Similarity Matrix - Embeddings')
axes[0, 1].set_xlabel('Sample Index')
axes[0, 1].set_ylabel('Sample Index')
plt.colorbar(im2, ax=axes[0, 1])

# Scatter plot: Cosine similarity vs Euclidean distance
# Sample a subset for visualization
n_sample_vis = min(10000, len(cosine_sim_values))
sample_idx_vis = np.random.choice(len(cosine_sim_values), n_sample_vis, replace=False)

axes[0, 2].scatter(cosine_sim_values[sample_idx_vis], euclidean_dist_values[sample_idx_vis], 
                   alpha=0.5, s=1)
axes[0, 2].set_xlabel('Cosine Similarity')
axes[0, 2].set_ylabel('Euclidean Distance')
axes[0, 2].set_title(f'Cosine Sim vs Euclidean Dist\n(corr={correlation:.3f})')

# Distribution of Euclidean distances
axes[1, 0].hist(euclidean_dist_values, bins=100, alpha=0.7, density=True, color='green')
axes[1, 0].set_title('Euclidean Distance Distribution - Embeddings')
axes[1, 0].set_xlabel('Distance')
axes[1, 0].set_ylabel('Density')

# Distribution of cosine similarities (for comparison)
axes[1, 1].hist(cosine_sim_values, bins=100, alpha=0.7, density=True, color='blue')
axes[1, 1].set_title('Cosine Similarity Distribution - Embeddings')
axes[1, 1].set_xlabel('Similarity')
axes[1, 1].set_ylabel('Density')

# Combined distributions (normalized)
axes[1, 2].hist(cosine_distance_values, bins=100, alpha=0.6, density=True, 
                color='blue', label='Cosine Distance')
axes[1, 2].hist(euclidean_normalized, bins=100, alpha=0.6, density=True, 
                color='green', label='Euclidean (normalized)')
axes[1, 2].set_title('Distance Distributions Comparison')
axes[1, 2].set_xlabel('Distance')
axes[1, 2].set_ylabel('Density')
axes[1, 2].legend()

plt.tight_layout()
plt.savefig('embedding_similarity_distance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved as 'embedding_similarity_distance_comparison.png'")


In [26]:
# Create KL divergence matrix function
from scipy.spatial.distance import jensenshannon
from scipy.stats import entropy

def create_kl_divergence_matrix_efficient(adata, sample_size=None, epsilon=1e-10):
    """
    Create KL divergence matrix for the data
    
    Parameters:
    -----------
    adata : AnnData
        The data to compute KL divergences for
    sample_size : int, optional
        If provided, randomly sample this many vectors to make computation feasible
    epsilon : float
        Small value to add for numerical stability
    
    Returns:
    --------
    kl_div_matrix : np.ndarray
        KL divergence matrix
    indices : np.ndarray
        Indices of sampled vectors (if sampling was used)
    """
    # Get the data matrix
    X = adata.X.toarray() if hasattr(adata.X, 'toarray') else adata.X
    
    # Sample if requested
    if sample_size is not None and sample_size < X.shape[0]:
        print(f"Sampling {sample_size} vectors from {X.shape[0]} total vectors")
        indices = np.random.choice(X.shape[0], sample_size, replace=False)
        X_sample = X[indices]
        print(f"Sample shape: {X_sample.shape}")
    else:
        X_sample = X
        indices = np.arange(X.shape[0])
        print(f"Using all {X.shape[0]} vectors")
    
    # Convert to probability distributions
    # First, handle negative values by shifting to positive
    X_shifted = X_sample - X_sample.min(axis=1, keepdims=True) + epsilon
    
    # Normalize to probability distributions (sum to 1)
    X_prob = X_shifted / X_shifted.sum(axis=1, keepdims=True)
    
    print("Computing KL divergence matrix...")
    n_samples = X_prob.shape[0]
    kl_div_matrix = np.zeros((n_samples, n_samples))
    
    # Compute KL divergence for each pair
    for i in range(n_samples):
        if i % 100 == 0:
            print(f"Processing sample {i}/{n_samples}")
        for j in range(n_samples):
            if i == j:
                kl_div_matrix[i, j] = 0.0  # KL(P||P) = 0
            else:
                # KL(P||Q) = sum(P * log(P/Q))
                p = X_prob[i]
                q = X_prob[j]
                # Add small epsilon to q to avoid division by zero
                q_safe = q + epsilon
                kl_div = entropy(p, q_safe)
                kl_div_matrix[i, j] = kl_div
    
    print(f"KL divergence matrix shape: {kl_div_matrix.shape}")
    return kl_div_matrix, indices

# Compute KL divergence for embeddings
print("=== KL DIVERGENCE FOR EMBEDDINGS ===")
kl_div_emb_sample, indices_kl_emb = create_kl_divergence_matrix_efficient(adata_real_emb, sample_size=500)


=== KL DIVERGENCE FOR EMBEDDINGS ===
Sampling 500 vectors from 49389 total vectors
Sample shape: (500, 2058)
Computing KL divergence matrix...
Processing sample 0/500
Processing sample 100/500
Processing sample 200/500
Processing sample 300/500
Processing sample 400/500
KL divergence matrix shape: (500, 500)


In [27]:
# Analyze the KL divergence matrix for embeddings
print("\n=== KL DIVERGENCE MATRIX STATISTICS (EMBEDDINGS) ===")
print(f"Matrix shape: {kl_div_emb_sample.shape}")
print(f"Mean KL divergence: {kl_div_emb_sample.mean():.6f}")
print(f"Std KL divergence: {kl_div_emb_sample.std():.6f}")
print(f"Min KL divergence: {kl_div_emb_sample.min():.6f}")
print(f"Max KL divergence: {kl_div_emb_sample.max():.6f}")

# Check for infinite values
n_inf = np.isinf(kl_div_emb_sample).sum()
n_nan = np.isnan(kl_div_emb_sample).sum()
print(f"Number of infinite values: {n_inf}")
print(f"Number of NaN values: {n_nan}")

# Diagonal should be 0.0 (KL(P||P) = 0)
diagonal_values_kl = np.diag(kl_div_emb_sample)
print(f"Diagonal values (should be ~0.0): mean={diagonal_values_kl.mean():.6f}, std={diagonal_values_kl.std():.6f}")

# Off-diagonal KL divergences (excluding self-divergence)
mask_kl = ~np.eye(kl_div_emb_sample.shape[0], dtype=bool)
off_diagonal_kl = kl_div_emb_sample[mask_kl]

# Remove infinite and NaN values for statistics
off_diagonal_kl_finite = off_diagonal_kl[np.isfinite(off_diagonal_kl)]
print(f"Off-diagonal KL divergences (finite only): mean={off_diagonal_kl_finite.mean():.6f}, std={off_diagonal_kl_finite.std():.6f}")
print(f"Number of finite off-diagonal values: {len(off_diagonal_kl_finite)} out of {len(off_diagonal_kl)}")

# Look at distribution of KL divergences
if len(off_diagonal_kl_finite) > 0:
    print(f"Embedding off-diagonal KL divergence percentiles:")
    for p in percentiles:
        val = np.percentile(off_diagonal_kl_finite, p)
        print(f"  {p:2d}%: {val:.6f}")

    # Find most and least divergent pairs
    sorted_off_diag_kl_idx = np.argsort(off_diagonal_kl_finite)
    print(f"\nMost similar embedding pair (lowest KL divergence): {off_diagonal_kl_finite[sorted_off_diag_kl_idx[0]]:.6f}")
    print(f"Most divergent embedding pair (highest finite KL divergence): {off_diagonal_kl_finite[sorted_off_diag_kl_idx[-1]]:.6f}")



=== KL DIVERGENCE MATRIX STATISTICS (EMBEDDINGS) ===
Matrix shape: (500, 500)
Mean KL divergence: inf
Std KL divergence: nan
Min KL divergence: -0.000567
Max KL divergence: inf
Number of infinite values: 105592
Number of NaN values: 0
Diagonal values (should be ~0.0): mean=0.000000, std=0.000000
Off-diagonal KL divergences (finite only): mean=0.000364, std=0.000339
Number of finite off-diagonal values: 143908 out of 249500
Embedding off-diagonal KL divergence percentiles:
   1%: -0.000329
   5%: -0.000179
  10%: -0.000076
  25%: 0.000125
  50%: 0.000353
  75%: 0.000591
  90%: 0.000808
  95%: 0.000927
  99%: 0.001223

Most similar embedding pair (lowest KL divergence): -0.000567
Most divergent embedding pair (highest finite KL divergence): 0.002168


/home/dhruvgautam/miniconda/envs/pertsets/lib/python3.12/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [ ]:
# Compare KL divergence with cosine similarity and Euclidean distance
print("\n=== COMPARISON: KL DIVERGENCE vs COSINE SIMILARITY vs EUCLIDEAN DISTANCE ===")

# We need to get the same sample indices for fair comparison
# Let's use the KL divergence sample size (500) and match the indices
cosine_sim_kl_sample, _ = create_cosine_similarity_matrix_efficient(adata_real_emb, sample_size=500)
euclidean_dist_kl_sample, _ = create_euclidean_distance_matrix_efficient(adata_real_emb, sample_size=500)

# Get off-diagonal values for all three metrics
mask_kl_comp = ~np.eye(kl_div_emb_sample.shape[0], dtype=bool)

cosine_sim_kl_values = cosine_sim_kl_sample[mask_kl_comp]
euclidean_dist_kl_values = euclidean_dist_kl_sample[mask_kl_comp]
kl_div_kl_values = kl_div_emb_sample[mask_kl_comp]

# Remove infinite/NaN values for correlation analysis
finite_mask = np.isfinite(kl_div_kl_values)
cosine_finite = cosine_sim_kl_values[finite_mask]
euclidean_finite = euclidean_dist_kl_values[finite_mask]
kl_finite = kl_div_kl_values[finite_mask]

print(f"Number of finite comparisons: {len(kl_finite)} out of {len(kl_div_kl_values)}")

if len(kl_finite) > 1000:  # Need sufficient data for correlation
    # Calculate correlations
    corr_kl_cosine = np.corrcoef(kl_finite, cosine_finite)[0, 1]
    corr_kl_euclidean = np.corrcoef(kl_finite, euclidean_finite)[0, 1]
    
    # Convert cosine similarity to distance for better comparison
    cosine_dist_finite = 1 - cosine_finite
    corr_kl_cosine_dist = np.corrcoef(kl_finite, cosine_dist_finite)[0, 1]
    
    print(f"Correlation between KL divergence and cosine similarity: {corr_kl_cosine:.6f}")
    print(f"Correlation between KL divergence and cosine distance: {corr_kl_cosine_dist:.6f}")
    print(f"Correlation between KL divergence and Euclidean distance: {corr_kl_euclidean:.6f}")
    
    print(f"\nSummary statistics for comparison:")
    print(f"KL divergence:     mean={kl_finite.mean():.6f}, std={kl_finite.std():.6f}")
    print(f"Cosine similarity: mean={cosine_finite.mean():.6f}, std={cosine_finite.std():.6f}")
    print(f"Cosine distance:   mean={cosine_dist_finite.mean():.6f}, std={cosine_dist_finite.std():.6f}")
    print(f"Euclidean distance: mean={euclidean_finite.mean():.6f}, std={euclidean_finite.std():.6f}")
else:
    print("Insufficient finite values for correlation analysis")


In [ ]:
# Create visualization comparing all three distance/similarity metrics
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Heatmap of KL divergence matrix (embeddings)
# Clip extreme values for better visualization
kl_clipped = np.clip(kl_div_emb_sample, 0, np.percentile(kl_div_emb_sample[np.isfinite(kl_div_emb_sample)], 95))
im1 = axes[0, 0].imshow(kl_clipped, cmap='viridis', aspect='auto')
axes[0, 0].set_title('KL Divergence Matrix - Embeddings\n(clipped at 95th percentile)')
axes[0, 0].set_xlabel('Sample Index')
axes[0, 0].set_ylabel('Sample Index')
plt.colorbar(im1, ax=axes[0, 0])

# Heatmap of cosine similarity matrix for comparison
im2 = axes[0, 1].imshow(cosine_sim_kl_sample, cmap='viridis', aspect='auto')
axes[0, 1].set_title('Cosine Similarity Matrix - Embeddings')
axes[0, 1].set_xlabel('Sample Index')
axes[0, 1].set_ylabel('Sample Index')
plt.colorbar(im2, ax=axes[0, 1])

# Heatmap of Euclidean distance matrix for comparison
im3 = axes[0, 2].imshow(euclidean_dist_kl_sample, cmap='viridis', aspect='auto')
axes[0, 2].set_title('Euclidean Distance Matrix - Embeddings')
axes[0, 2].set_xlabel('Sample Index')
axes[0, 2].set_ylabel('Sample Index')
plt.colorbar(im3, ax=axes[0, 2])

# Scatter plot: KL divergence vs Cosine similarity (if we have enough finite values)
if len(kl_finite) > 1000:
    n_sample_vis = min(5000, len(kl_finite))
    sample_idx_vis = np.random.choice(len(kl_finite), n_sample_vis, replace=False)
    
    axes[0, 3].scatter(cosine_finite[sample_idx_vis], kl_finite[sample_idx_vis], 
                       alpha=0.5, s=1)
    axes[0, 3].set_xlabel('Cosine Similarity')
    axes[0, 3].set_ylabel('KL Divergence')
    axes[0, 3].set_title(f'Cosine Sim vs KL Divergence\n(corr={corr_kl_cosine:.3f})')
else:
    axes[0, 3].text(0.5, 0.5, 'Insufficient\nfinite values', ha='center', va='center', transform=axes[0, 3].transAxes)
    axes[0, 3].set_title('KL vs Cosine (insufficient data)')

# Distribution plots
axes[1, 0].hist(kl_finite, bins=50, alpha=0.7, density=True, color='red')
axes[1, 0].set_title('KL Divergence Distribution')
axes[1, 0].set_xlabel('KL Divergence')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_yscale('log')

axes[1, 1].hist(cosine_finite, bins=50, alpha=0.7, density=True, color='blue')
axes[1, 1].set_title('Cosine Similarity Distribution')
axes[1, 1].set_xlabel('Cosine Similarity')
axes[1, 1].set_ylabel('Density')

axes[1, 2].hist(euclidean_finite, bins=50, alpha=0.7, density=True, color='green')
axes[1, 2].set_title('Euclidean Distance Distribution')
axes[1, 2].set_xlabel('Euclidean Distance')
axes[1, 2].set_ylabel('Density')

# Combined comparison plot
if len(kl_finite) > 1000:
    # Normalize all metrics to [0,1] for comparison
    kl_norm = (kl_finite - kl_finite.min()) / (kl_finite.max() - kl_finite.min())
    cosine_dist_norm = (cosine_dist_finite - cosine_dist_finite.min()) / (cosine_dist_finite.max() - cosine_dist_finite.min())
    euclidean_norm = (euclidean_finite - euclidean_finite.min()) / (euclidean_finite.max() - euclidean_finite.min())
    
    axes[1, 3].hist(kl_norm, bins=50, alpha=0.5, density=True, color='red', label='KL Divergence')
    axes[1, 3].hist(cosine_dist_norm, bins=50, alpha=0.5, density=True, color='blue', label='Cosine Distance')
    axes[1, 3].hist(euclidean_norm, bins=50, alpha=0.5, density=True, color='green', label='Euclidean Distance')
    axes[1, 3].set_title('Normalized Distance Distributions')
    axes[1, 3].set_xlabel('Normalized Distance')
    axes[1, 3].set_ylabel('Density')
    axes[1, 3].legend()
else:
    axes[1, 3].text(0.5, 0.5, 'Insufficient\nfinite values', ha='center', va='center', transform=axes[1, 3].transAxes)

plt.tight_layout()
plt.savefig('kl_divergence_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved as 'kl_divergence_comparison.png'")


In [18]:
def create_reference_pca_anndatas(adata_reference, adata_query, n_components=50, scale=True):
    """
    Fit PCA on reference dataset and apply to both reference and query datasets
    
    Parameters:
    -----------
    adata_reference : AnnData
        Reference dataset to fit PCA on (adata_real)
    adata_query : AnnData  
        Query dataset to transform using reference PCA (adata_pred)
    n_components : int
        Number of PCA components
    scale : bool
        Whether to standardize the data
    
    Returns:
    --------
    tuple of (adata_reference_pca, adata_query_pca)
    """
    # Get data matrices
    X_ref = adata_reference.X.copy()
    X_query = adata_query.X.copy()
    
    # Convert to dense if sparse
    if hasattr(X_ref, 'toarray'):
        X_ref = X_ref.toarray()
    if hasattr(X_query, 'toarray'):
        X_query = X_query.toarray()
    
    # Fit scaler on reference data only
    if scale:
        scaler = StandardScaler()
        X_ref_scaled = scaler.fit_transform(X_ref)
        X_query_scaled = scaler.transform(X_query)  # Use reference scaler
    else:
        X_ref_scaled = X_ref
        X_query_scaled = X_query
    
    # Fit PCA on reference data only
    pca = PCA(n_components=n_components)
    X_ref_pca = pca.fit_transform(X_ref_scaled)
    
    # Transform query data using reference PCA
    X_query_pca = pca.transform(X_query_scaled)
    
    # Create var dataframe
    var_names = [f'PC{i+1}' for i in range(n_components)]
    new_var = pd.DataFrame(index=var_names)
    new_var['explained_variance_ratio'] = pca.explained_variance_ratio_
    new_var['explained_variance'] = pca.explained_variance_
    
    # Create AnnData objects
    adata_ref_pca = ad.AnnData(X=X_ref_pca, obs=adata_reference.obs.copy(), var=new_var.copy())
    adata_query_pca = ad.AnnData(X=X_query_pca, obs=adata_query.obs.copy(), var=new_var.copy())
    
    # Store PCA info
    pca_info = {
        'components': pca.components_,
        'explained_variance_ratio': pca.explained_variance_ratio_,
        'explained_variance': pca.explained_variance_,
        'n_components': n_components,
        'scaled': scale,
        'fitted_on': 'reference'
    }
    
    adata_ref_pca.uns['pca'] = pca_info
    adata_query_pca.uns['pca'] = pca_info
    
    return adata_ref_pca, adata_query_pca

# If you want both datasets in the same PC space:
# adata_pred_pca, adata_real_pca = create_matched_pca_anndatas(adata_pred, adata_real, n_components=50)

In [24]:
adata_real_pca, adata_pred_pca = create_reference_pca_anndatas(adata_real, adata_pred, n_components=128)
compute_perturbation_ranking_score(adata_pred_pca, adata_real_pca, pert_col="gene", ctrl_pert="non-targeting")

/tmp/ipykernel_3168865/238431976.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3168865/238431976.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()


0.26922095956017794

In [27]:
adata_real_emb_pca, adata_pred_emb_pca = create_reference_pca_anndatas(adata_real_emb, adata_pred_emb, n_components=128)
print(compute_perturbation_ranking_score(adata_pred_emb_pca, adata_real_emb_pca, pert_col="gene", ctrl_pert="non-targeting"))
print(compute_perturbation_ranking_score_rev(adata_pred_emb_pca, adata_real_emb_pca, pert_col="gene", ctrl_pert="non-targeting"))

/tmp/ipykernel_3168865/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3168865/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()


0.27592112028306665


/tmp/ipykernel_3168865/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3168865/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()


0.33161949564590765


In [37]:
compute_perturbation_ranking_score(adata_pred_emb_min, adata_real_emb_min, pert_col="gene", ctrl_pert="non-targeting")

/tmp/ipykernel_3168865/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3168865/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()


0.338534223449348

In [32]:
compute_perturbation_ranking_score(adata_pred, adata_real, pert_col="gene", ctrl_pert="non-targeting")

/tmp/ipykernel_3168865/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3168865/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()


0.2652777834006048

In [3]:
adata_pred_mean = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/replogle_filtered_globalsimplesum/rpe1/eval_final.ckpt/adata_pred.h5ad')
adata_real_mean = ad.read_h5ad('/large_storage/ctc/userspace/aadduri/preprint/replogle_filtered_globalsimplesum/rpe1/eval_final.ckpt/adata_real.h5ad')

In [ ]:
compute_perturbation_ranking_score(adata_pred_mean, adata_real_mean), compute_perturbation_ranking_score_rev(adata_pred_mean, adata_real_mean)

/tmp/ipykernel_3173493/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3173493/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()


0.24906379929476255

In [37]:
compute_perturbation_ranking_score(adata_pred_mean, adata_real_mean), compute_perturbation_ranking_score_rev(adata_pred_mean, adata_real_mean)

/tmp/ipykernel_3173493/2320155802.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3173493/2320155802.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()


ValueError: `x` and `y` must have the same length along `axis`.

In [9]:
compute_perturbation_ranking_score(adata_pred_emb, adata_real_emb), compute_perturbation_ranking_score_rev(adata_pred_emb, adata_real_emb)

/tmp/ipykernel_3173493/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3173493/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3173493/856578241.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = adata_df.groupby("pert").mean()
/tmp/ipykernel_3173493/856578241.py:4: Futu

(0.29083150816367015, 0.3407811051419179)